In [2]:
import pandas as pd
from uncertainties import ufloat, unumpy
from uncertainties.umath import log10
import numpy as np

# Asegurarse de que tienes las columnas en tu DataFrame
# df['H_ALPHA_FLUX'], df['H_ALPHA_FLUX_ERR'], df['OIII_5007_FLUX'], df['OIII_5007_FLUX_ERR'],
# df['H_BETA_FLUX'], df['H_BETA_FLUX_ERR'], df['NII_6584_FLUX'], df['NII_6584_FLUX_ERR']

df = pd.read_csv('measurement_emission_lines.tsv', delimiter='\t')
df = df.drop(index=[8,9,10,11,12])

# Crear columnas con ufloat
df['H_ALPHA_FLUX_UN'] = df.apply(lambda row: ufloat(row['H_ALPHA_FLUX'], row['H_ALPHA_FLUX_ERR']), axis=1)
df['OIII_5007_FLUX_UN'] = df.apply(lambda row: ufloat(row['OIII_5007_FLUX'], row['OIII_5007_FLUX_ERR']), axis=1)
df['H_BETA_FLUX_UN'] = df.apply(lambda row: ufloat(row['H_BETA_FLUX'], row['H_BETA_FLUX_ERR']), axis=1)
df['NII_6584_FLUX_UN'] = df.apply(lambda row: ufloat(row['NII_6584_FLUX'], row['NII_6584_FLUX_ERR']), axis=1)

# Calcular las razones con incertidumbres
o3_hb_un = df['OIII_5007_FLUX_UN'] / df['H_BETA_FLUX_UN']
nii_ha_un = df['NII_6584_FLUX_UN'] / df['H_ALPHA_FLUX_UN']

# Calcular O3N2 = log10((OIII 5007 / Hbeta) / (NII 6584 / Halpha))
o3n2_un = unumpy.log10(o3_hb_un / nii_ha_un)

# Definir constantes con incertidumbres
const_o3n2 = ufloat(8.533, 0.012)
coef_o3n2 = ufloat(0.214, 0.012)

# Calcular 12 + log(O/H) usando O3N2
abund_o3n2_un = const_o3n2 - coef_o3n2 * o3n2_un

# Calcular N2 = log10(NII 6584 / Halpha)
n2_un = unumpy.log10(nii_ha_un)

# Definir constantes con incertidumbres
const_n2 = ufloat(8.743, 0.027)
coef_n2 = ufloat(0.462, 0.024)

# Calcular 12 + log(O/H) usando N2
abund_n2_un = const_n2 + coef_n2 * n2_un

# Añadir las columnas calculadas al DataFrame
df['O3N2'] = unumpy.nominal_values(o3n2_un)
df['O3N2_ERR'] = unumpy.std_devs(o3n2_un)
df['12_LOG_OH_O3N2'] = unumpy.nominal_values(abund_o3n2_un)
df['12_LOG_OH_O3N2_ERR'] = unumpy.std_devs(abund_o3n2_un)

df['N2'] = unumpy.nominal_values(n2_un)
df['N2_ERR'] = unumpy.std_devs(n2_un)
df['12_LOG_OH_N2'] = unumpy.nominal_values(abund_n2_un)
df['12_LOG_OH_N2_ERR'] = unumpy.std_devs(abund_n2_un)

# Mostrar el DataFrame con las nuevas columnas
print(df[['O3N2', 'O3N2_ERR', '12_LOG_OH_O3N2', '12_LOG_OH_O3N2_ERR', 'N2', 'N2_ERR', '12_LOG_OH_N2', '12_LOG_OH_N2_ERR']])


       O3N2  O3N2_ERR  12_LOG_OH_O3N2  12_LOG_OH_O3N2_ERR        N2    N2_ERR  \
0  2.301985  0.112996        8.040375            0.038624 -1.558421  0.111177   
1  2.154682  0.126795        8.071898            0.039355 -1.457303  0.123748   
2  1.655616  0.063629        8.178698            0.026910 -1.185229  0.058070   
3  2.027372  0.247328        8.099142            0.059475 -1.391182  0.246533   
4  1.691130  0.173196        8.171098            0.043927 -1.270309  0.172368   
5  1.628656  0.074348        8.184468            0.027912 -1.202156  0.071916   
6  2.106784  0.080028        8.082148            0.032809 -1.469700  0.075061   
7  1.362967  0.120515        8.241325            0.032812 -1.130775  0.117968   

   12_LOG_OH_N2  12_LOG_OH_N2_ERR  
0      8.023009          0.069037  
1      8.069726          0.072256  
2      8.195424          0.047517  
3      8.100274          0.121723  
4      8.156117          0.089443  
5      8.187604          0.051627  
6      8.063999   

In [4]:
import pandas as pd
from uncertainties import ufloat, unumpy
from uncertainties.umath import log10

df = pd.read_csv('measurement_emission_lines.tsv', delimiter='\t')
df = df.drop(index=[8,9,10,11,12])

# Crear columnas con ufloat en tu DataFrame
df['H_ALPHA_FLUX_UN'] = df.apply(lambda row: ufloat(row['H_ALPHA_FLUX'], row['H_ALPHA_FLUX_ERR']), axis=1)
df['OIII_5007_FLUX_UN'] = df.apply(lambda row: ufloat(row['OIII_5007_FLUX'], row['OIII_5007_FLUX_ERR']), axis=1)
df['H_BETA_FLUX_UN'] = df.apply(lambda row: ufloat(row['H_BETA_FLUX'], row['H_BETA_FLUX_ERR']), axis=1)
df['NII_6584_FLUX_UN'] = df.apply(lambda row: ufloat(row['NII_6584_FLUX'], row['NII_6584_FLUX_ERR']), axis=1)

# Calcular las razones con incertidumbres
df['OIII_HB_UN'] = df.apply(lambda row: row['OIII_5007_FLUX_UN'] / row['H_BETA_FLUX_UN'], axis=1)
df['NII_HA_UN'] = df.apply(lambda row: row['NII_6584_FLUX_UN'] / row['H_ALPHA_FLUX_UN'], axis=1)

# Calcular O3N2 = log10((OIII 5007 / Hbeta) / (NII 6584 / Halpha))
df['O3N2_UN'] = df.apply(lambda row: log10(row['OIII_HB_UN'] / row['NII_HA_UN']), axis=1)

# Definir constantes con incertidumbres para O3N2
const_o3n2 = ufloat(8.533, 0.012)
coef_o3n2 = ufloat(0.214, 0.012)

# Calcular 12 + log(O/H) usando O3N2
df['12_LOG_OH_O3N2_UN'] = df.apply(lambda row: const_o3n2 - coef_o3n2 * row['O3N2_UN'], axis=1)

# Calcular N2 = log10(NII 6584 / Halpha)
df['N2_UN'] = df['NII_HA_UN'].apply(log10)

# Definir constantes con incertidumbres para N2
const_n2 = ufloat(8.743, 0.027)
coef_n2 = ufloat(0.462, 0.024)

# Calcular 12 + log(O/H) usando N2
df['12_LOG_OH_N2_UN'] = df.apply(lambda row: const_n2 + coef_n2 * row['N2_UN'], axis=1)

# Añadir las columnas calculadas al DataFrame
df['O3N2'] = df['O3N2_UN'].apply(unumpy.nominal_values)
df['O3N2_ERR'] = df['O3N2_UN'].apply(unumpy.std_devs)
df['12_LOG_OH_O3N2'] = df['12_LOG_OH_O3N2_UN'].apply(unumpy.nominal_values)
df['12_LOG_OH_O3N2_ERR'] = df['12_LOG_OH_O3N2_UN'].apply(unumpy.std_devs)

df['N2'] = df['N2_UN'].apply(unumpy.nominal_values)
df['N2_ERR'] = df['N2_UN'].apply(unumpy.std_devs)
df['12_LOG_OH_N2'] = df['12_LOG_OH_N2_UN'].apply(unumpy.nominal_values)
df['12_LOG_OH_N2_ERR'] = df['12_LOG_OH_N2_UN'].apply(unumpy.std_devs)

# Mostrar el DataFrame con las nuevas columnas
print(df[['O3N2', 'O3N2_ERR', '12_LOG_OH_O3N2', '12_LOG_OH_O3N2_ERR', 'N2', 'N2_ERR', '12_LOG_OH_N2', '12_LOG_OH_N2_ERR']])


       O3N2  O3N2_ERR  12_LOG_OH_O3N2  12_LOG_OH_O3N2_ERR        N2    N2_ERR  \
0  2.301985  0.112996        8.040375            0.038624 -1.558421  0.111177   
1  2.154682  0.126795        8.071898            0.039355 -1.457303  0.123748   
2  1.655616  0.063629        8.178698            0.026910 -1.185229  0.058070   
3  2.027372  0.247328        8.099142            0.059475 -1.391182  0.246533   
4  1.691130  0.173196        8.171098            0.043927 -1.270309  0.172368   
5  1.628656  0.074348        8.184468            0.027912 -1.202156  0.071916   
6  2.106784  0.080028        8.082148            0.032809 -1.469700  0.075061   
7  1.362967  0.120515        8.241325            0.032812 -1.130775  0.117968   

   12_LOG_OH_N2  12_LOG_OH_N2_ERR  
0      8.023009          0.069037  
1      8.069726          0.072256  
2      8.195424          0.047517  
3      8.100274          0.121723  
4      8.156117          0.089443  
5      8.187604          0.051627  
6      8.063999   

In [5]:
import pandas as pd
from uncertainties import ufloat, unumpy

df = pd.read_csv('measurement_emission_lines.tsv', delimiter='\t')
df = df.drop(index=[8,9,10,11,12])


# Crear columnas con ufloat en tu DataFrame
df['H_ALPHA_FLUX_UN'] = df.apply(lambda row: ufloat(row['H_ALPHA_FLUX'], row['H_ALPHA_FLUX_ERR']), axis=1)
df['OIII_5007_FLUX_UN'] = df.apply(lambda row: ufloat(row['OIII_5007_FLUX'], row['OIII_5007_FLUX_ERR']), axis=1)
df['H_BETA_FLUX_UN'] = df.apply(lambda row: ufloat(row['H_BETA_FLUX'], row['H_BETA_FLUX_ERR']), axis=1)
df['NII_6584_FLUX_UN'] = df.apply(lambda row: ufloat(row['NII_6584_FLUX'], row['NII_6584_FLUX_ERR']), axis=1)

# Calcular las razones con incertidumbres
df['OIII_HB_UN'] = df['OIII_5007_FLUX_UN'] / df['H_BETA_FLUX_UN']
df['NII_HA_UN'] = df['NII_6584_FLUX_UN'] / df['H_ALPHA_FLUX_UN']

# Calcular O3N2 = log10((OIII 5007 / Hbeta) / (NII 6584 / Halpha))
df['O3N2_UN'] = unumpy.log10(df['OIII_HB_UN'] / df['NII_HA_UN'])

# Definir constantes con incertidumbres para O3N2
const_o3n2 = ufloat(8.533, 0.012)
coef_o3n2 = ufloat(0.214, 0.012)

# Calcular 12 + log(O/H) usando O3N2
df['12_LOG_OH_O3N2_UN'] = const_o3n2 - coef_o3n2 * df['O3N2_UN']

# Calcular N2 = log10(NII 6584 / Halpha)
df['N2_UN'] = unumpy.log10(df['NII_HA_UN'])

# Definir constantes con incertidumbres para N2
const_n2 = ufloat(8.743, 0.027)
coef_n2 = ufloat(0.462, 0.024)

# Calcular 12 + log(O/H) usando N2
df['12_LOG_OH_N2_UN'] = const_n2 + coef_n2 * df['N2_UN']

# Añadir las columnas calculadas al DataFrame
df['O3N2'] = unumpy.nominal_values(df['O3N2_UN'])
df['O3N2_ERR'] = unumpy.std_devs(df['O3N2_UN'])
df['12_LOG_OH_O3N2'] = unumpy.nominal_values(df['12_LOG_OH_O3N2_UN'])
df['12_LOG_OH_O3N2_ERR'] = unumpy.std_devs(df['12_LOG_OH_O3N2_UN'])

df['N2'] = unumpy.nominal_values(df['N2_UN'])
df['N2_ERR'] = unumpy.std_devs(df['N2_UN'])
df['12_LOG_OH_N2'] = unumpy.nominal_values(df['12_LOG_OH_N2_UN'])
df['12_LOG_OH_N2_ERR'] = unumpy.std_devs(df['12_LOG_OH_N2_UN'])

# Mostrar el DataFrame con las nuevas columnas
print(df[['O3N2', 'O3N2_ERR', '12_LOG_OH_O3N2', '12_LOG_OH_O3N2_ERR', 'N2', 'N2_ERR', '12_LOG_OH_N2', '12_LOG_OH_N2_ERR']])


       O3N2  O3N2_ERR  12_LOG_OH_O3N2  12_LOG_OH_O3N2_ERR        N2    N2_ERR  \
0  2.301985  0.112996        8.040375            0.038624 -1.558421  0.111177   
1  2.154682  0.126795        8.071898            0.039355 -1.457303  0.123748   
2  1.655616  0.063629        8.178698            0.026910 -1.185229  0.058070   
3  2.027372  0.247328        8.099142            0.059475 -1.391182  0.246533   
4  1.691130  0.173196        8.171098            0.043927 -1.270309  0.172368   
5  1.628656  0.074348        8.184468            0.027912 -1.202156  0.071916   
6  2.106784  0.080028        8.082148            0.032809 -1.469700  0.075061   
7  1.362967  0.120515        8.241325            0.032812 -1.130775  0.117968   

   12_LOG_OH_N2  12_LOG_OH_N2_ERR  
0      8.023009          0.069037  
1      8.069726          0.072256  
2      8.195424          0.047517  
3      8.100274          0.121723  
4      8.156117          0.089443  
5      8.187604          0.051627  
6      8.063999   